In [23]:
import os
import json
import numpy as np
from rdkit import Chem

from matplotlib import pyplot as plt
from sklearn import metrics
from itertools import islice
import pandas as pd
import re

In [2]:
gnps_dir = "G:\\Dev\\Data\\GNPS"
json_file_dir = "G:\\Dev\\trees_gnps"
fragments_occurences = "G:\\Dev\\fragment_occurences.csv"
losses_occurences = "G:\\Dev\\losses_occurences.csv"

path = "G:\\Dev\\CSIFingerID\\"
sample_fingerprints_legend_path = path + "fingerprints.csv"
output_file_dir = path + "all_sirius_output_minibatches"

In [3]:
fragments_df = pd.read_csv(fragments_occurences, names=["formula", "occurences"])
fragments_df = fragments_df.set_index("formula")
losses_df = pd.read_csv(losses_occurences, names=["formula", "occurences"])
losses_df = losses_df.set_index("formula")


In [4]:
losses_df.index = [index + "_loss" for index in losses_df.index]
print(losses_df.index)
combined_index = fragments_df.index.union(losses_df.index)
print(combined_index)

Index(['CO_loss', 'H2O_loss', 'C2H2_loss', 'CH3_loss', 'CHN_loss', 'CH4_loss',
       'H3N_loss', 'H2_loss', 'C2H4_loss', 'C2H2O_loss',
       ...
       'C3H8S_loss', 'OP_loss', 'CH2N3_loss', 'C7H7NO3_loss', 'O2P_loss',
       'C2H5NOS_loss', 'C10H4N3_loss', 'C9H17O3_loss', 'C3H7N4_loss',
       'C4H6NO2_loss'],
      dtype='object', length=892)
Index(['BrH_loss', 'Br_loss', 'C10', 'C10H10', 'C10H10ClN', 'C10H10ClNO',
       'C10H10ClNO2', 'C10H10F2O2', 'C10H10FN', 'C10H10N',
       ...
       'O2_loss', 'O3S_loss', 'O3_loss', 'O4_loss', 'O5_loss', 'OP_loss',
       'OS_loss', 'O_loss', 'S2_loss', 'S_loss'],
      dtype='object', length=5508)


In [5]:
def read_tree(filepath, fragment_formula, losses_formula):
    fragments = {}
    losses = {}
    all_fragments = {}
    
    with open(filepath, 'r') as f:
        data = json.load(f)

    for fragment in data['fragments']:
        if "intensity" in fragment:
            all_fragments[(fragment['molecularFormula'])] = float(fragment['intensity'])
            if fragment['molecularFormula'] in fragment_formula:
                fragments[(fragment['molecularFormula'])] = float(fragment['intensity'])

    for loss in data['losses']:
        loss_index = loss['molecularFormula'] + "_loss"
        if loss_index in losses_formula and loss["source"] in all_fragments and loss["target"] in all_fragments:
            losses[loss_index] = (all_fragments[loss["source"]] + all_fragments[loss["target"]]) /2 
                
    return fragments, losses

# Load a master file containing CDK fingerprints for all molecules.
# Each molecules CDK bit set is added as a 320 element array to a Pandas dataframe.
def load_fingerprints_master(path, number_of_rows=0):
    BITS = 307  # Total number of bits in fingerprint

    fp_all = np.loadtxt(path, dtype="U25", skiprows=number_of_rows) # Get master file as numpy array of Strings
    fp_ids = np.unique(fp_all[:, 0]) # Trim duplicate filename rows, store unique filenames

    # Construct empty Pandas dataframe of correct size.
    # Number of rows is equal to the number of unique molecules (found in fp_ids).
    fingerprints = pd.DataFrame(0, index = fp_ids, columns=range(BITS), dtype=int)

    # Populate the dataframe using each molecule's filename to place data in the correct row.
    for row in fp_all:
        fingerprints.at[row[0], int(row[1])] = int(row[2])

    # Convert populated dataframe into a numpy array for use as output by neural networks.
    return fingerprints

# Load the names of all substructures included in the CDK fingerprint in the correct order
# This is used for boxplots, when performance metrics for individual substructures are calculated.
def load_fingerprint_legend():
    fingerprint_legend = []
    # Open file containing substructure names.
    with open(fingerprints_names_path, 'r') as f:
        # Add each name to the list of substructure names.
        lines = list(islice(f, 0, None))
        for line in lines:
            fingerprint_legend.append(line[:-1])
    return fingerprint_legend

In [6]:
def dict_is_empty(sample_dict):
    return len(sample_dict.keys()) == 0

def get_mol_names(path_dir):
    return [file[:-3] for file in os.listdir(path_dir)]

In [7]:
component_set = set()

for formula in combined_index:
    pure_formula = formula.split('_loss')
    components = re.findall('[A-Z][^A-Z\d]*', pure_formula[0])
    for component in components:
        component_set.add(component)

mol_mass = {'C': 12.00000000000, 'H': 1.00782503214, 'O': 15.99491462210, 'N': 14.00307400524,
            'P': 30.97376151200, 'S': 31.97207069000, 'Cl': 34.96885271000, 'I': 126.904468, 'Br': 78.9183376,
            'Si': 27.9769265327, 'F': 18.99840320500, 'D': 2.01410177800}

ion_mass = {'H': 1.007276, 'Na': 22.989218, 'K': 38.963158}

def get_number(string):
    num = 1
    num_in_string = re.findall("[\d]+", string)
    if len(num_in_string) > 0:
        num = int(num_in_string[0])
    return num

def get_string(string):
    return re.findall("[^\d]+", string)[0]  

def get_molecular_mass(formula):
    mass = 0
    segments = re.findall('[A-Z][a-z]*[0-9]*', formula)
    for segment in segments:
        mass += mol_mass[get_string(segment)] * get_number(segment)
    
    return mass

fragment_mass_to_formula_dict = {}
loss_mass_to_formula_dict = {}

for formula in fragments_df.index:
    mass = get_molecular_mass(formula)
    mass_int, decimals = str(mass).split(".")
    if mass_int not in fragment_mass_to_formula_dict:
        fragment_mass_to_formula_dict[int(mass_int)] = {}
    fragment_mass_to_formula_dict[int(mass_int)][round(mass, 6)] = formula
    
for formula in losses_df.index:
    pure_formula = formula.split('_loss')
    mass = get_molecular_mass(pure_formula[0])
    mass_int, decimals = str(mass).split(".")
    if mass_int not in loss_mass_to_formula_dict:
        loss_mass_to_formula_dict[int(mass_int)] = {}
    loss_mass_to_formula_dict[int(mass_int)][round(mass, 6)] = pure_formula[0]

In [8]:
def subtract_formula(a, b):
    string = ""
    a_segments = re.findall('[A-Z][a-z]*[0-9]*', a)
    b_segments = re.findall('[A-Z][a-z]*[0-9]*', b)

    a_segment_count = [(get_string(a_segment), get_number(a_segment)) for a_segment in a_segments]

    for b_segment in b_segments:
        for index, a_segment in enumerate(a_segment_count):
            if a_segment[0] == get_string(b_segment):
                a_segment_count[index] = (a_segment[0], a_segment[1] - get_number(b_segment))

    for component, amount in a_segment_count:
        if amount != 0:
            if amount == 1:
                amount = ""
            string += component + str(amount)

    return string

In [9]:
def convert_lines_to_list(lines):
    mass_intensity_list = []
    for line in lines:
        if ' ' in line:  # Only lines with mass and intensity values have a space. Ignores label/blank lines
            split_line = line.split()
            mass = float(split_line[0])
            intensity = float(split_line[1])
            mass_intensity_list.append((mass, intensity))
            
    return mass_intensity_list

def count_in_vector(mass_intensities, ionization):
    count = 0
    for mass, intensity in mass_intensities:
        non_ionized_mass = mass - ionization
        mass_int, decimals = str(non_ionized_mass).split(".")
        if int(mass_int) in fragment_mass_to_formula_dict:
            for full_mass, formula in fragment_mass_to_formula_dict[int(mass_int)].items():
                if full_mass-0.0001*full_mass <= non_ionized_mass <= full_mass+0.0001*full_mass:
                    count += 1
    return count

def assign_to_tree(gnps_dir):
    mol_ids = get_mol_names(gnps_dir)
    tree_intensities = pd.DataFrame(0.0, index=mol_ids, columns=combined_index, dtype=float)
    
#     filepath = "G:\\Dev\\Data\\GNPS\\CCMSLIB00000001563.ms"
    
#     with open(filepath, 'r') as f:
#         ionization_entries_count = []
#         unsplit_lines = list(islice(f, 9, None))
#         mass_intensity_list = convert_lines_to_list(unsplit_lines)
#         for ion in ion_mass.keys():
#             ionization_entries_count.append((ion, count_in_vector(mass_intensity_list, ion_mass[ion])))
#         ion_with_most_entries = max(ionization_entries_count, key=lambda x:x[1])[0]
#         ionization = ion_mass[ion_with_most_entries]

#         for mass, intensity in mass_intensity_list:
#             non_ionized_mass = mass - ionization
#             mass_int, decimals = str(non_ionized_mass).split(".")
#             if int(mass_int) in fragment_mass_to_formula_dict:
#                 for full_mass, formula in fragment_mass_to_formula_dict[int(mass_int)].items():
#                     if full_mass-0.0001*full_mass <= non_ionized_mass <= full_mass+0.0001*full_mass:
#                         tree_intensities.at['CCMSLIB00000001563', fragment_mass_to_formula_dict[int(mass_int)][full_mass]] = float(intensity)
#                         assigned_formula_intensity.append((fragment_mass_to_formula_dict[int(mass_int)][full_mass], intensity))
                        
#         assigned_formula_intensity.reverse() # highest to smallest
                
#         peak_differences = [(subtract_formula(formula_intensity_i[0], formula_intensity_j[0]) + "_loss", 
#                              (formula_intensity_i[1] + formula_intensity_i[1]) / 2.0 ) 
#                             for i, formula_intensity_i in enumerate(assigned_formula_intensity)
#                             for j, formula_intensity_j in enumerate(assigned_formula_intensity) 
#                             if i != j and j > i and subtract_formula(formula_intensity_i[0], formula_intensity_j[0]) + "_loss" in losses_df.index]
        
#         for formula, intensity in peak_differences:
#             tree_intensities.at['CCMSLIB00000001563', formula] = float(intensity)
    
    for file in os.listdir(gnps_dir):
        assigned_formula_intensity = []
        filepath = os.path.join(gnps_dir, file)
        print(file)
        with open(filepath, 'r') as f:
            ionization_entries_count = []
            unsplit_lines = list(islice(f, 9, None))
            mass_intensity_list = convert_lines_to_list(unsplit_lines)
            for ion in ion_mass.keys():
                ionization_entries_count.append((ion, count_in_vector(mass_intensity_list, ion_mass[ion])))
            ion_with_most_entries = max(ionization_entries_count, key=lambda x:x[1])[0]
            ionization = ion_mass[ion_with_most_entries]

            for mass, intensity in mass_intensity_list:
                non_ionized_mass = mass - ionization
                mass_int, decimals = str(non_ionized_mass).split(".")
                if int(mass_int) in fragment_mass_to_formula_dict:
                    for full_mass, formula in fragment_mass_to_formula_dict[int(mass_int)].items():
                        if full_mass-0.0001*full_mass <= non_ionized_mass <= full_mass+0.0001*full_mass:
                            tree_intensities.at[file[:-3], fragment_mass_to_formula_dict[int(mass_int)][full_mass]] = float(intensity)
                            assigned_formula_intensity.append((fragment_mass_to_formula_dict[int(mass_int)][full_mass], intensity))

            assigned_formula_intensity.reverse() # highest to smallest

            peak_differences = [(subtract_formula(formula_intensity_i[0], formula_intensity_j[0]) + "_loss", 
                                 (formula_intensity_i[1] + formula_intensity_i[1]) / 2.0 ) 
                                for i, formula_intensity_i in enumerate(assigned_formula_intensity)
                                for j, formula_intensity_j in enumerate(assigned_formula_intensity) 
                                if i != j 
                                and j > i
                                and len(re.findall('[A-Z][a-z]*[0-9]*', formula_intensity_i[0])) >= len(re.findall('[A-Z][a-z]*[0-9]*', formula_intensity_j[0]))
                                and subtract_formula(formula_intensity_i[0], formula_intensity_j[0]) + "_loss" in losses_df.index]

            for formula, intensity in peak_differences:
                tree_intensities.at[file[:-3], formula] = float(intensity)

    return tree_intensities

assigned_tree = assign_to_tree(gnps_dir)

CCMSLIB00000001548.ms
CCMSLIB00000001549.ms
CCMSLIB00000001550.ms
CCMSLIB00000001555.ms
CCMSLIB00000001563.ms
CCMSLIB00000001565.ms
CCMSLIB00000001566.ms
CCMSLIB00000001568.ms
CCMSLIB00000001569.ms
CCMSLIB00000001570.ms
CCMSLIB00000001572.ms
CCMSLIB00000001574.ms
CCMSLIB00000001576.ms
CCMSLIB00000001581.ms
CCMSLIB00000001590.ms
CCMSLIB00000001598.ms
CCMSLIB00000001600.ms
CCMSLIB00000001601.ms
CCMSLIB00000001602.ms
CCMSLIB00000001603.ms
CCMSLIB00000001604.ms
CCMSLIB00000001606.ms
CCMSLIB00000001607.ms
CCMSLIB00000001608.ms
CCMSLIB00000001609.ms
CCMSLIB00000001615.ms
CCMSLIB00000001616.ms
CCMSLIB00000001617.ms
CCMSLIB00000001621.ms
CCMSLIB00000001622.ms
CCMSLIB00000001623.ms
CCMSLIB00000001624.ms
CCMSLIB00000001625.ms
CCMSLIB00000001631.ms
CCMSLIB00000001633.ms
CCMSLIB00000001634.ms
CCMSLIB00000001635.ms
CCMSLIB00000001637.ms
CCMSLIB00000001638.ms
CCMSLIB00000001641.ms
CCMSLIB00000001642.ms
CCMSLIB00000001643.ms
CCMSLIB00000001645.ms
CCMSLIB00000001646.ms
CCMSLIB00000001650.ms
CCMSLIB000

KeyboardInterrupt: 

In [11]:
assigned_tree_path = "G:\\Dev\\Data\\assigned_tree_final.pkl"

In [298]:
assigned_tree.to_pickle(assigned_tree_path)

In [19]:
assigned_tree = pd.read_pickle(assigned_tree_path)
print(load_assigned_tree)

                    BrH_loss  Br_loss           C10  C10H10  C10H10ClN  \
CCMSLIB00000001548       0.0      0.0      0.000000     0.0        0.0   
CCMSLIB00000001549       0.0      0.0      0.000000     0.0        0.0   
CCMSLIB00000001550       0.0      0.0      0.000000     0.0        0.0   
CCMSLIB00000001555       0.0      0.0      0.000000     0.0        0.0   
CCMSLIB00000001563       0.0      0.0      0.000000     0.0        0.0   
CCMSLIB00000001565       0.0      0.0      0.000000     0.0        0.0   
CCMSLIB00000001566       0.0      0.0      0.000000     0.0        0.0   
CCMSLIB00000001568       0.0      0.0      0.000000     0.0        0.0   
CCMSLIB00000001569       0.0      0.0      0.000000     0.0        0.0   
CCMSLIB00000001570       0.0      0.0      0.000000     0.0        0.0   
CCMSLIB00000001572       0.0      0.0      0.000000     0.0        0.0   
CCMSLIB00000001574       0.0      0.0      0.000000     0.0        0.0   
CCMSLIB00000001576       0.0      0.0 

In [13]:
from keras.layers import Input, Dense
from keras.models import Model,Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from keras.optimizers import SGD

def baseline_model(x_train_formula, x_train_fingerprints):
    class_model = Sequential()
    class_model.add(Dense(2500, input_dim=x_train_formula.shape[1], kernel_initializer='normal', activation='relu'))
    class_model.add(Dense(1200,kernel_initializer='normal',activation = 'relu'))
    class_model.add(Dense(600,kernel_initializer='normal',activation = 'relu'))
    class_model.add(Dense(x_train_fingerprints.shape[1],kernel_initializer='normal',activation = 'sigmoid'))
    class_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return class_model

Using TensorFlow backend.


In [14]:
fingerprints_path = "G:\\Dev\\Data\\1000\\GNPS Python Master\\Final Fingerprints.txt"

In [15]:
fingerprints = load_fingerprints_master(fingerprints_path)

for idx, value in fingerprints.astype(bool).sum(axis=0).iteritems():
    if value < 10:
        fingerprints.drop(columns=[idx], inplace=True)
        
fingerprints.sort_index(inplace=True)

print(fingerprints)

                    0    1    2    3    4    5    7    8    11   12  ...   \
CCMSLIB00000001548    1    1    1    0    0    0    0    0    1    0 ...    
CCMSLIB00000001549    1    1    0    0    1    0    0    0    1    0 ...    
CCMSLIB00000001550    0    0    0    0    1    0    0    0    0    0 ...    
CCMSLIB00000001555    1    1    1    0    0    0    1    0    1    0 ...    
CCMSLIB00000001563    1    1    1    0    0    0    0    0    0    0 ...    
CCMSLIB00000001565    1    1    1    0    1    1    0    0    0    0 ...    
CCMSLIB00000001566    1    1    1    0    0    0    0    0    0    0 ...    
CCMSLIB00000001568    1    1    1    0    0    0    0    0    1    1 ...    
CCMSLIB00000001569    1    1    1    0    1    1    0    0    0    0 ...    
CCMSLIB00000001570    1    1    1    0    0    0    0    0    1    1 ...    
CCMSLIB00000001572    1    1    1    0    0    0    0    0    1    1 ...    
CCMSLIB00000001574    1    1    0    0    0    0    0    0    0    0 ...    

In [20]:
def compute_auc(indexes, true, pred):
    auc_scores = []
    
    for i, index in enumerate(indexes):
        nonzero_vals = np.count_nonzero(true[:, i]) # Count number of nonzero values
        if nonzero_vals > 0 and nonzero_vals < true[:, i].size: # If there are no 1s or no 0s, can't compute.
            fp_true = true[:, i]
            fp_pred = pred[:, i]
            score = metrics.roc_auc_score(fp_true, fp_pred)
            auc_scores.append((index, score))
        else:
            auc_scores.append((index, 0.0))
            
    print("Compute AUC done")
    return auc_scores

def compute_f1(indexes, true, pred):
    f1_scores = []
    
    for i, index in enumerate(indexes):
        nonzero_vals = np.count_nonzero(true[:, i]) # Count number of nonzero values
        if nonzero_vals > 0 and nonzero_vals < true[:, i].size: # If there are no 1s or no 0s, can't compute.
            fp_true = true[:, i]
            fp_pred = pred[:, i]
            score = metrics.f1_score(fp_true, fp_pred, average='micro')
            f1_scores.append((index, score))
        else:
            f1_scores.append((index, 0.0))
            
    print("Compute F1 done")
    return f1_scores

In [25]:
def get_tree_loss_training(fingerprints, tree_with_loss, train):
    train_tree = tree_with_loss[tree_with_loss.index.isin(train.index)]
    train_tree.sort_index(inplace=True)
    x_train_formula = np.log(train_tree.values+1)
    
    train_fingerprints = fingerprints[fingerprints.index.isin(train_tree.index)]
    train_fingerprints.sort_index(inplace=True)
    x_train_fingerprints = train_fingerprints.values
    
    return x_train_formula, x_train_fingerprints

def get_tree_loss_validation(fingerprints, tree_with_loss, validate):
    validate_tree = tree_with_loss[tree_with_loss.index.isin(validate.index)]
    validate_tree.sort_index(inplace=True)
    x_validate_formula = np.log(validate_tree.values+1)
    
    validate_fingerprints = fingerprints[fingerprints.index.isin(validate_tree.index)]
    validate_fingerprints.sort_index(inplace=True)
    x_validate_fingerprints = validate_fingerprints.values
    
    return x_validate_formula, x_validate_fingerprints

def get_tree_loss_test(fingerprints, tree_with_loss, test):
    test_tree = tree_with_loss[tree_with_loss.index.isin(test.index)]
    test_tree.sort_index(inplace=True)
    x_test_formula = np.log(test_tree.values+1)
    
    test_fingerprints = fingerprints[fingerprints.index.isin(test_tree.index)]
    test_fingerprints.sort_index(inplace=True)
    x_test_fingerprints = test_fingerprints.values
    
    return x_test_formula, x_test_fingerprints

def run_experiments(fingerprints, tree_with_losses):
    epochs = 100

    train, validate, test = np.split(tree_with_losses.sample(frac=1), [int(.6*len(tree_with_losses)), int(.8*len(tree_with_losses))])
    
    x_train_formula_with_loss, x_train_fingerprints_with_loss = get_tree_loss_training(fingerprints, tree_with_losses, train)
    x_validate_formula_with_loss, x_validate_fingerprints_with_loss = get_tree_loss_validation(fingerprints, tree_with_losses, validate)
    x_test_formula_with_loss, x_test_fingerprints_with_loss = get_tree_loss_test(fingerprints, tree_with_losses, test)
    
    mod = baseline_model(x_train_formula_with_loss, x_train_fingerprints_with_loss)
    history = mod.fit(x_train_formula_with_loss, x_train_fingerprints_with_loss, epochs=epochs, 
                        validation_data=(x_validate_formula_with_loss,x_validate_fingerprints_with_loss), verbose=0)
    
    predicted = mod.predict(x_test_formula_with_loss)
    stats = compute_auc(fingerprints.columns.tolist(), x_test_fingerprints_with_loss, predicted)
    
    prediction = np.zeros((predicted.shape))
    prediction[predicted > 0.5] = 1
    f1_stats = compute_f1(fingerprints.columns.tolist(), x_test_fingerprints_with_loss, prediction)
    
    return stats, f1_stats


In [ ]:
# experiment_auc_path = "G://Dev//Data//tree_vs_tree_with_shifts_experiment//"

for i in range(1):
    print(i)
    auc_stats, f1_stats = run_experiments(fingerprints, assigned_tree)
    print(auc_stats)
    print()
    print(f1_stats)
    
#     base_exp_aucs_path = experiment_auc_path + "experiment_{}_aucs.csv".format(i)
#     f1_scores_path = experiment_auc_path + "f1_scores_experiment_{}.csv".format(i)
    
#     with open(base_exp_aucs_path, 'w') as f:
#         for i, auc_score in enumerate(base_stats):
#             fingerprint_index, auc = auc_score
#             f.write(str(fingerprint_index) + "," + str(auc) + "," + str(comparison_stats[i][1]) + "\n")
    
#     with open(f1_scores_path, 'w') as f:
#         for i, f1_score in enumerate(base_f1_stats):
#             fingerprint_index, f1 = f1_score
#             f.write(str(fingerprint_index) + "," + str(f1) + "," + str(comparison_f1_stats[i][1]) + "\n")

0
